# Word distribution visualize
To visualize the word distribution of generated story text.


In [ ]:
from __future__ import division
import os
import nltk
import numpy as np
from matplotlib import pyplot as plt
from nltk.stem import WordNetLemmatizer
from collections import Counter

import sys
sys.path.append('../')
from rule_mining import utils as ut
from rule_mining import config as cfg

class TextProcesser(object):
    def __init__(self):
        if not os.path.exists(cfg.txtdata_file):
            get_txtdata()
        txtdata = ut.load_json(cfg.txtdata_file)

        self.words2id = txtdata['words2id']
        self.id2words = txtdata['id2words']
        self.words = txtdata['words']
        self.word_tag = txtdata['word_tag']
        self.disabled_list = txtdata['disabled_list']
        self.word_count = txtdata['word_count']
        self.wnl = WordNetLemmatizer()

    def get_tag(self, word):
        if word in self.word_tag:
            tag = self.word_tag[word]
        else:
            tag = nltk.pos_tag([word])[0][1]
        return tag

    def lemmatize(self, word):
        tag = self.get_tag(word)
        if tag.startswith('NN'):
            lemm = self.wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            lemm = self.wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            lemm = self.wnl.lemmatize(word, pos='a')
        elif tag.startswith('R'):
            lemm = self.wnl.lemmatize(word, pos='r')
        else:
            lemm = word

        return lemm

In [ ]:
def show_word_distr(tp, predictions):
    counter = Counter()
    print 'len(predictions)=%d' % len(predictions)
    total_list = []
    for album_id, prediction in predictions.items():
        word_list = ut.split_sentence(prediction[0])
        for word in word_list:
            if word not in tp.disabled_list:
                counter.update([word])
                if word in tp.words2id:
                    total_list.append(tp.words2id[word])

    print counter.most_common(30)
    data = sorted(counter.values(), reverse=True)[:50]
    datasum = sum(data)
    data = [value/datasum * 100 for value in data]
    plt.figure()
    axes = plt.gca()
    axes.set_ylim([0, 16])
    plt.bar(x=range(len(data)), height=data, width=1)
    plt.show()

In [ ]:
def word_count():
    tp = TextProcesser()
    
    predictions_list = ['/home/lijiacheng/AREL-master/data/save/IRL-XE-ss-init/',
                        '/home/lijiacheng/VSCMR/data/save/default/']
    for model in predictions_list:
        predictions_file = os.path.join(model, "prediction_test.json")
        if os.path.exists(predictions_file):
            predictions = ut.load_json(predictions_file)
            ut.printn("Processing {}".format(model))
        else:
            continue
        show_word_distr(tp,predictions)
word_count()